# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
#impts
import os
import json
import requests
import gradio as gr
from dotenv import load_dotenv
from typing import List

In [ ]:
#Env set up

load_dotenv()

#local-i have ollama installed on my pc
OLLAMA_URL = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434/v1/completions")
OLLAMA_MODEL = os.getenv("LOCAL_MODEL_NAME", "llama3.2")

#cloude have anthropic api key
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
ANTHROPIC_MODEL = "claude-sonnet-4-5-20250929"  

#system prompt
SYSTEM_PROMPT = """You are a helpful AI assistant with expertise in Python and software engineering.
Explain code clearly and concisely. If asked about tools, respond only with accurate outputs."""

In [ ]:
#tool

def get_ticket_price(city: str) -> dict:
    prices = {"Nairobi": 120, "Paris": 800, "Tokyo": 950, "New York": 700}
    return {"destination": city, "price": prices.get(city, "unknown")}


def detect_and_run_tool(user_message: str):
    if user_message.strip().startswith("/ticket"):
        parts = user_message.strip().split()
        if len(parts) > 1:
            city = parts[1]
            return True, get_ticket_price(city)
    return False, None


In [ ]:
#Call fncs

def ask_ollama(prompt: str) -> str:
    try:
        payload = {"model": OLLAMA_MODEL, "prompt": prompt, "stream": False}
        r = requests.post(OLLAMA_URL, json=payload, timeout=120)
        r.raise_for_status()
        data = r.json()
        return data.get("choices", [{}])[0].get("text", "").strip()
    except Exception as e:
        return f"[Ollama error: {e}]"

In [ ]:

def ask_anthropic(prompt: str) -> str:
    import anthropic
    client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
    try:
        response = client.messages.create(
            model=ANTHROPIC_MODEL,
            system=SYSTEM_PROMPT,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=512,
        )
        return "".join(block.text for block in response.content if block.type == "text")
    except Exception as e:
        return f"[Anthropic error: {e}]"


In [ ]:
#chat LOgic
def chat_fn(user_message: str, history: List[List[str]], model_choice: str):
    tool_used, tool_output = detect_and_run_tool(user_message)
    if tool_used:
        reply = f"Tool executed. Result:\n{json.dumps(tool_output, indent=2)}"
    else:
        if model_choice == "anthropic":
            reply = ask_anthropic(user_message)
        else:
            prompt = f"{SYSTEM_PROMPT}\n\nUser: {user_message}\nAssistant:"
            reply = ask_ollama(prompt)

    history.append([user_message, reply])
    return history

In [ ]:
#Gradio UI
def main():
    with gr.Blocks(title="Week 2 QA Prototype") as demo:
        gr.Markdown("### Week 2 — Full Q/A Prototype\nSupports Anthropic & Ollama models, plus a `/ticket <city>` tool.")

        # Message input comes first
        msg = gr.Textbox(
            placeholder="Ask your question or type /ticket Nairobi",
            label="Your message",
            lines=2,
            autofocus=True
        )

        # Model selection and send button next
        with gr.Row():
            model_choice = gr.Radio(["ollama", "anthropic"], value="ollama", label="Model")
            send = gr.Button("Send", variant="primary")

        # Chatbot area
        chatbot = gr.Chatbot(label="Conversation", height=400)

        # Wrapper 
        def wrapped_chat_fn(user_message, history, model_choice):
            updated_history = chat_fn(user_message, history, model_choice)
            return updated_history, gr.update(value="")

        # Send button and Enter key submission
        send.click(wrapped_chat_fn, inputs=[msg, chatbot, model_choice], outputs=[chatbot, msg])
        msg.submit(wrapped_chat_fn, inputs=[msg, chatbot, model_choice], outputs=[chatbot, msg])

    demo.launch(server_name="0.0.0.0", share=False)


if __name__ == "__main__":
    main()
